In [93]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch

In [94]:
# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [95]:
def load_and_process_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    dialogues = [line.strip().split('|||') for line in lines if line.strip()]
    # Process each dialogue pair
    processed_examples = []
    for dialogue in dialogues:
        customer, agent = dialogue
        combined_text = customer + tokenizer.eos_token + agent + tokenizer.eos_token
        tokenized = tokenizer(combined_text, truncation=True, max_length=512, padding='max_length')
        # Ensure the 'labels' are correctly set for loss computation
        tokenized['labels'] = tokenized['input_ids'].copy()
        processed_examples.append(tokenized)

    # Creating a DataFrame
    df = pd.DataFrame(processed_examples)

    train_df, eval_df = train_test_split(df, test_size=0.1)
    train_dataset = Dataset.from_pandas(train_df)
    eval_dataset = Dataset.from_pandas(eval_df)

    return train_dataset, eval_dataset

In [96]:
# Custom collate function to ensure dynamic padding
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids_padded = pad_sequence([torch.tensor(seq) for seq in input_ids], 
                                    batch_first=True, padding_value=tokenizer.pad_token_id)
    labels_padded = pad_sequence([torch.tensor(seq) for seq in labels], 
                                 batch_first=True, padding_value=tokenizer.pad_token_id)

    return {'input_ids': input_ids_padded, 'labels': labels_padded}


In [97]:
# Load and process the dataset
train_dataset, eval_dataset = load_and_process_data('nsp_fine_tuning_dataset.txt')


In [98]:
import distutils
print("Distutils module is found!")


Distutils module is found!


In [99]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=custom_collate_fn,  # Use the custom collate function
)


In [100]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "gpt2_nsp_trainer.ipynb"


In [101]:
import wandb
wandb.login(key="a9433891c8b370b61ad36f3b5c379d41d79ff4dd")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [102]:
# Train the model
trainer.train()

  2%|▏         | 10/540 [01:24<1:14:24,  8.42s/it]

{'loss': 12.043, 'grad_norm': 243.45953369140625, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}


  4%|▎         | 20/540 [02:50<1:15:16,  8.69s/it]

{'loss': 8.8652, 'grad_norm': 243.82521057128906, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}


  6%|▌         | 30/540 [04:19<1:14:09,  8.72s/it]

...

100%|██████████| 540/540 [1:27:10<00:00,  8.75s/it]

{'loss': 0.0543, 'grad_norm': 1.0225006341934204, 'learning_rate': 0.0, 'epoch': 3.0}


                                                   
100%|██████████| 540/540 [1:28:15<00:00,  9.81s/it]

{'eval_loss': 0.06480729579925537, 'eval_runtime': 65.5515, 'eval_samples_per_second': 0.61, 'eval_steps_per_second': 0.305, 'epoch': 3.0}
{'train_runtime': 5295.6435, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.102, 'train_loss': 0.549343987637096, 'epoch': 3.0}


TrainOutput(global_step=540, training_loss=0.549343987637096, metrics={'train_runtime': 5295.6435, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.102, 'train_loss': 0.549343987637096, 'epoch': 3.0})

In [103]:
# Save the fine-tuned model
model_path = './fine_tuned_gpt2'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./fine_tuned_gpt2\\tokenizer_config.json',
 './fine_tuned_gpt2\\special_tokens_map.json',
 './fine_tuned_gpt2\\vocab.json',
 './fine_tuned_gpt2\\merges.txt',
 './fine_tuned_gpt2\\added_tokens.json')